In [1]:
# Import libraries
import pandas as pd
import os

In [ ]:
data = pd.read_csv('all_proposed_amendments.csv', encoding='ISO-8859-1')

#print(data.columns)
""" Returned:
Index(['identifier', 'source_code', 'source_citation', 'source_index_number',
       'title_or_description_from_source', 'date_approximation', 'year',
       'month', 'day', 'congress', 'congressional_session',
       'joint_resolution_chamber', 'joint_resolution_number', 'sponsor_name',
       'sponsor_state_or_territory', 'committee_of_referral', 'last_modified'],
      dtype='object')
"""

"\nIndex(['identifier', 'source_code', 'source_citation', 'source_index_number',\n       'title_or_description_from_source', 'date_approximation', 'year',\n       'month', 'day', 'congress', 'congressional_session',\n       'joint_resolution_chamber', 'joint_resolution_number', 'sponsor_name',\n       'sponsor_state_or_territory', 'committee_of_referral', 'last_modified'],\n      dtype='object')\n"

In [3]:
data.head()

,identifier,source_code,source_citation,source_index_number,title_or_description_from_source,date_approximation,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,last_modified
0,us-nara-amending-america-000001,A,"Ames, Herman V. The Proposed Amendments to the...",1,Reservation of nondelegated powers,circa,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00
1,us-nara-amending-america-000002,A,"Ames, Herman V. The Proposed Amendments to the...",2,Apportionment of Representatives,circa,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00
2,us-nara-amending-america-000003,A,"Ames, Herman V. The Proposed Amendments to the...",3,Restriction on Federal control over election o...,circa,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00
3,us-nara-amending-america-000004,A,"Ames, Herman V. The Proposed Amendments to the...",4,Restriction upon the levying of direct taxes,circa,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00
4,us-nara-amending-america-000005,A,"Ames, Herman V. The Proposed Amendments to the...",5,Commercial monopolies prohibited,circa,1788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00


In [4]:
data.tail()

,identifier,source_code,source_citation,source_index_number,title_or_description_from_source,date_approximation,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,last_modified
11792,us-nara-amending-america-011793,G,United States Congress. Congress.gov: Legislat...,NaN,A joint resolution proposing an amendment to t...,NaN,2013.0,6.0,18.0,113.0,NaN,Senate Joint Resolution,19.0,"Udall, Tom",New Mexico,Judiciary,2016-02-25T17:29:59.732-05:00
11793,us-nara-amending-america-011794,G,United States Congress. Congress.gov: Legislat...,NaN,A joint resolution proposing a balanced budget...,NaN,2013.0,7.0,11.0,113.0,NaN,Senate Joint Resolution,20.0,"Udall, Tom",New Mexico,Judiciary,2016-02-25T17:29:59.732-05:00
11794,us-nara-amending-america-011795,G,United States Congress. Congress.gov: Legislat...,NaN,A joint resolution proposing an amendment to t...,NaN,2013.0,10.0,11.0,113.0,NaN,Senate Joint Resolution,25.0,"Paul, Rand",Kentucky,Judiciary,2016-02-25T17:29:59.732-05:00
11795,us-nara-amending-america-011796,G,United States Congress. Congress.gov: Legislat...,NaN,A joint resolution proposing an amendment to t...,NaN,2014.0,3.0,13.0,113.0,NaN,Senate Joint Resolution,34.0,"Enzi, Michael",Wyoming,Judiciary,2016-02-25T17:29:59.732-05:00
11796,us-nara-amending-america-011797,G,United States Congress. Congress.gov: Legislat...,NaN,A joint resolution proposing an amendment to t...,NaN,2014.0,6.0,4.0,113.0,NaN,Senate Joint Resolution,37.0,"Graham, Lindsey",South Carolina,Judiciary,2016-02-25T17:29:59.732-05:00


There is a lot of duplicates within these proposed amendments since they go through multiple stages within Congress and HoS and could take months, I will filter out duplicates within their 'title_or_description_from_source' column and keep the latest ones

In [ ]:
# Filter out duplicates in the 'title_or_description_from_source' column, keep the latest entry
data_unique = data.copy().drop_duplicates(subset='title_or_description_from_source', keep='last')
print(f'Dropped {data.shape[0] - data_unique.shape[0]} duplicate entries based on title_or_description_from_source.')

(11797, 17)
Dropped 7066 duplicate entries based on title_or_description_from_source.


In [6]:
# Search the data_unique column "title_or_description_from_source" for the word "abortion"
abortion_data = data_unique[data_unique['title_or_description_from_source'].str.contains('abortion', case=False, na=False)]
print(abortion_data.shape)
abortion_data

(22, 17)


,identifier,source_code,source_citation,source_index_number,title_or_description_from_source,date_approximation,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,last_modified
8811,us-nara-amending-america-008812,E,"Davis, Richard. Proposed Amendments to the Con...",1648,Prohibit abortion,NaN,1974.0,5.0,30.0,93.0,NaN,House Joint Resolution,1041.0,Roncallo,New York,NaN,2016-02-25T17:29:59.732-05:00
8929,us-nara-amending-america-008930,E,"Davis, Richard. Proposed Amendments to the Con...",1766,"Power of States to allow, regulate or prohibit...",NaN,1975.0,1.0,16.0,94.0,NaN,House Joint Resolution,97.0,Dingell,Michigan,NaN,2016-02-25T17:29:59.732-05:00
8987,us-nara-amending-america-008988,E,"Davis, Richard. Proposed Amendments to the Con...",1824,"Power of states to allow, regulate or prohibit...",NaN,1975.0,2.0,27.0,94.0,NaN,House Joint Resolution,261.0,Whitehurst,Virginia,NaN,2016-02-25T17:29:59.732-05:00
9055,us-nara-amending-america-009056,E,"Davis, Richard. Proposed Amendments to the Con...",1891,Prohibit abortion after heartbeat starts,NaN,1975.0,6.0,5.0,94.0,NaN,House Joint Resolution,485.0,Eshleman,Pennsylvania,NaN,2016-02-25T17:29:59.732-05:00
9064,us-nara-amending-america-009065,E,"Davis, Richard. Proposed Amendments to the Con...",1900,Power of states toåäóæregulate or prohibit ab...,NaN,1975.0,6.0,24.0,94.0,NaN,House Joint Resolution,527.0,Hungate,Missouri,NaN,2016-02-25T17:29:59.732-05:00
9074,us-nara-amending-america-009075,E,"Davis, Richard. Proposed Amendments to the Con...",1910,"Power of states to allow, regulate or prohibit...",NaN,1975.0,7.0,22.0,94.0,NaN,House Joint Resolution,584.0,Whitehurst,Virginia,NaN,2016-02-25T17:29:59.732-05:00
9177,us-nara-amending-america-009178,E,"Davis, Richard. Proposed Amendments to the Con...",2010,Abortion prohibited; protection of unborn chil...,NaN,1977.0,1.0,24.0,95.0,NaN,Senate Joint Resolution,15.0,Garn,Utah,NaN,2016-02-25T17:29:59.732-05:00
9188,us-nara-amending-america-009189,E,"Davis, Richard. Proposed Amendments to the Con...",2021,Power of States to regulate abortion,NaN,1977.0,3.0,10.0,95.0,NaN,Senate Joint Resolution,38.0,Scott,Virginia,NaN,2016-02-25T17:29:59.732-05:00
9199,us-nara-amending-america-009200,E,"Davis, Richard. Proposed Amendments to the Con...",2032,Right to life; abortion prohibited,NaN,1977.0,9.0,28.0,95.0,NaN,Senate Joint Resolution,84.0,Hatch,Utah,NaN,2016-02-25T17:29:59.732-05:00
9470,us-nara-amending-america-009471,E,"Davis, Richard. Proposed Amendments to the Con...",2303,State jurisdiction over abortion,NaN,1977.0,12.0,15.0,95.0,NaN,House Joint Resolution,680.0,Slack,West Virginia,NaN,2016-02-25T17:29:59.732-05:00


In [7]:
# Search for amendments related to prohibition
prohibition_data = data_unique[data_unique['title_or_description_from_source'].str.contains('prohibition', case=False, na=False)]
print(prohibition_data.shape)
prohibition_data

(27, 17)


,identifier,source_code,source_citation,source_index_number,title_or_description_from_source,date_approximation,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,last_modified
762,us-nara-amending-america-000763,A,"Ames, Herman V. The Proposed Amendments to the...",701,Prohibition of State bank notes,NaN,1840.0,2.0,24.0,26.0,1,NaN,NaN,Buchanan,Pennsylvania,NaN,2016-02-25T17:29:59.732-05:00
1102,us-nara-amending-america-001103,A,"Ames, Herman V. The Proposed Amendments to the...",985,Prohibition of slavery (The XIII Amendment),NaN,1864.0,2.0,10.0,38.0,1,NaN,NaN,Trumball,NaN,Judiciary,2016-02-25T17:29:59.732-05:00
1117,us-nara-amending-america-001118,A,"Ames, Herman V. The Proposed Amendments to the...",998,Prohibition of slavery,NaN,1864.0,4.0,8.0,38.0,1,NaN,NaN,Sumner,Massachusetts,NaN,2016-02-25T17:29:59.732-05:00
1204,us-nara-amending-america-001205,A,"Ames, Herman V. The Proposed Amendments to the...",1067,Prohibition of compensation for slaves emancip...,NaN,1866.0,1.0,5.0,39.0,1,NaN,NaN,Williams,Oregon,NaN,2016-02-25T17:29:59.732-05:00
1249,us-nara-amending-america-001250,A,"Ames, Herman V. The Proposed Amendments to the...",1107,To empower congress to enforce on the States a...,NaN,1866.0,2.0,5.0,39.0,1,NaN,NaN,Williams,Pennsylvania,NaN,2016-02-25T17:29:59.732-05:00
1520,us-nara-amending-america-001521,A,"Ames, Herman V. The Proposed Amendments to the...",1329,Prohibition of appropriations to sectarian sch...,NaN,1870.0,4.0,18.0,41.0,2,NaN,NaN,Burdett,Missouri,NaN,2016-02-25T17:29:59.732-05:00
1666,us-nara-amending-america-001667,A,"Ames, Herman V. The Proposed Amendments to the...",1462,Prohibition of special legislation in certain ...,NaN,1878.0,1.0,21.0,45.0,2,NaN,NaN,Tipton,Illinois,Judiciary,2016-02-25T17:29:59.732-05:00
1754,us-nara-amending-america-001755,A,"Ames, Herman V. The Proposed Amendments to the...",1544,Prohibition of polygamy and bigamy,NaN,1882.0,1.0,9.0,47.0,1,NaN,NaN,Thomas,Illinois,Judiciary,2016-02-25T17:29:59.732-05:00
1819,us-nara-amending-america-001820,A,"Ames, Herman V. The Proposed Amendments to the...",1607,Prohibition of grants or loan of aid to corpor...,NaN,1884.0,1.0,8.0,48.0,1,NaN,NaN,Beach,New York,Judiciary,2016-02-25T17:29:59.732-05:00
1830,us-nara-amending-america-001831,A,"Ames, Herman V. The Proposed Amendments to the...",1616,Prohibition of liquor traffic,NaN,1884.0,1.0,16.0,48.0,1,NaN,NaN,Plumb,Kansas,Education and Labor,2016-02-25T17:29:59.732-05:00


In [8]:
# Search for rows that have the word "repeal" in the "title_or_description_from_source" column
amendment_data = data_unique[data_unique['title_or_description_from_source'].str.contains('repeal', case=False, na=False)]
print(amendment_data.shape)
amendment_data

(99, 17)


,identifier,source_code,source_citation,source_index_number,title_or_description_from_source,date_approximation,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,last_modified
1105,us-nara-amending-america-001106,A,"Ames, Herman V. The Proposed Amendments to the...",986b,Repeal of the fugitive slave clause,NaN,1864.0,2.0,17.0,38.0,1,NaN,NaN,Sumner,Massachusetts,NaN,2016-02-25T17:29:59.732-05:00
1144,us-nara-amending-america-001145,A,"Ames, Herman V. The Proposed Amendments to the...",1025,To repeal Corwin amendment,NaN,1864.0,2.0,8.0,NaN,NaN,NaN,NaN,Anthony,Rhode Island,NaN,2016-02-25T17:29:59.732-05:00
1189,us-nara-amending-america-001190,A,"Ames, Herman V. The Proposed Amendments to the...",1054,Provision prohibiting export duties repealed,circa,1865.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00
1928,us-nara-amending-america-001929,A,"Ames, Herman V. The Proposed Amendments to the...",1714,Prohibition of the repeal of general pension laws,NaN,1888.0,1.0,23.0,50.0,1,NaN,NaN,Peters,Kansas,Judiciary,2016-02-25T17:29:59.732-05:00
2264,us-nara-amending-america-002265,B,"Tansill, Charles C. Proposed Amendments to the...",309,To repeal Fifteenth Amendment [The Right of Bl...,NaN,1900.0,2.0,14.0,56.0,1,House Joint Resolution,174.0,Underwood,Alabama,NaN,2016-02-25T17:29:59.732-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11664,us-nara-amending-america-011665,G,United States Congress. Congress.gov: Legislat...,NaN,Proposing an amendment to the Constitution of ...,NaN,2011.0,5.0,12.0,112.0,NaN,House Joint Resolution,62.0,"Bishop, Rob",Utah,"Judiciary, Subcommittee on the Constitution",2016-02-25T17:29:59.732-05:00
11727,us-nara-amending-america-011728,G,United States Congress. Congress.gov: Legislat...,NaN,Proposing an amendment to the Constitution of ...,NaN,2013.0,1.0,4.0,113.0,NaN,House Joint Resolution,15.0,"Serrano, Jose E.",New York,"Judiciary, Subcommittee on the Constitution an...",2016-02-25T17:29:59.732-05:00
11771,us-nara-amending-america-011772,G,United States Congress. Congress.gov: Legislat...,NaN,Proposing an amendment to the Constitution of ...,NaN,2013.0,11.0,15.0,113.0,NaN,House Joint Resolution,104.0,"Bridenstine, Jim",Oklahoma,"Judiciary, Subcommittee on the Constitution an...",2016-02-25T17:29:59.732-05:00
11775,us-nara-amending-america-011776,G,United States Congress. Congress.gov: Legislat...,NaN,Proposing an amendment to the Constitution of ...,NaN,2014.0,5.0,21.0,113.0,NaN,House Joint Resolution,115.0,"Bishop, Rob",Utah,"Judiciary, Subcommittee on the Constitution an...",2016-02-25T17:29:59.732-05:00


In [9]:
# Search for rows that have parantheses in the 'title_or_description_from_source' column
parentheses_data = data_unique[data_unique['title_or_description_from_source'].str.contains(r'\(.*\)', na=False)]
print(parentheses_data.shape)
parentheses_data

(44, 17)


,identifier,source_code,source_citation,source_index_number,title_or_description_from_source,date_approximation,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,last_modified
323,us-nara-amending-america-000324,A,"Ames, Herman V. The Proposed Amendments to the...",321,Extent of jurisdiction. (The XI Amendment),NaN,1794.0,1.0,2.0,3.0,1,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00
329,us-nara-amending-america-000330,A,"Ames, Herman V. The Proposed Amendments to the...",327,Expiration of term (June 1),NaN,1795.0,3.0,3.0,3.0,2,NaN,NaN,Burr,New York,NaN,2016-02-25T17:29:59.732-05:00
1102,us-nara-amending-america-001103,A,"Ames, Herman V. The Proposed Amendments to the...",985,Prohibition of slavery (The XIII Amendment),NaN,1864.0,2.0,10.0,38.0,1,NaN,NaN,Trumball,NaN,Judiciary,2016-02-25T17:29:59.732-05:00
1283,us-nara-amending-america-001284,A,"Ames, Herman V. The Proposed Amendments to the...",1135,Civil rights: Citizenship (The XIV Amendment),circa,1866.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00
1466,us-nara-amending-america-001467,A,"Ames, Herman V. The Proposed Amendments to the...",1284,Suffrage. (The XV Amendment),NaN,1867.0,3.0,7.0,40.0,2,NaN,NaN,Henderson,Missouri,NaN,2016-02-25T17:29:59.732-05:00
1590,us-nara-amending-america-001591,A,"Ames, Herman V. The Proposed Amendments to the...",1386,Executive: Choice: election by the people in d...,NaN,1874.0,6.0,22.0,43.0,1,NaN,NaN,Smith,New York,Elections,2016-02-25T17:29:59.732-05:00
1735,us-nara-amending-america-001736,A,"Ames, Herman V. The Proposed Amendments to the...",1526,Certain United States officers elected by the ...,NaN,1881.0,12.0,10.0,47.0,1,NaN,NaN,Voorhees,Indiana,Judiciary,2016-02-25T17:29:59.732-05:00
1890,us-nara-amending-america-001891,A,"Ames, Herman V. The Proposed Amendments to the...",1676,New date for Inauguration Day (April 30),NaN,1886.0,3.0,15.0,49.0,1,NaN,NaN,Ingalls,Kansas,Privileges and Elections,2016-02-25T17:29:59.732-05:00
1895,us-nara-amending-america-001896,A,"Ames, Herman V. The Proposed Amendments to the...",1681,Executive and Legislative: New date for Inaugu...,NaN,1886.0,6.0,14.0,49.0,1,NaN,NaN,Hoar,Massachusetts,NaN,2016-02-25T17:29:59.732-05:00
2264,us-nara-amending-america-002265,B,"Tansill, Charles C. Proposed Amendments to the...",309,To repeal Fifteenth Amendment [The Right of Bl...,NaN,1900.0,2.0,14.0,56.0,1,House Joint Resolution,174.0,Underwood,Alabama,NaN,2016-02-25T17:29:59.732-05:00


In [10]:
# Search for rows that have the regex pattern "(The {string} Amendment)" in title_or_description_from_source
ratified_data = data_unique[data_unique['title_or_description_from_source'].str.contains(r'\(The .+ Amendment\)', case=False, na=False)]
print(ratified_data.shape)
ratified_data.head()

(4, 17)


,identifier,source_code,source_citation,source_index_number,title_or_description_from_source,date_approximation,year,month,day,congress,congressional_session,joint_resolution_chamber,joint_resolution_number,sponsor_name,sponsor_state_or_territory,committee_of_referral,last_modified
323,us-nara-amending-america-000324,A,"Ames, Herman V. The Proposed Amendments to the...",321,Extent of jurisdiction. (The XI Amendment),NaN,1794.0,1.0,2.0,3.0,1,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00
1102,us-nara-amending-america-001103,A,"Ames, Herman V. The Proposed Amendments to the...",985,Prohibition of slavery (The XIII Amendment),NaN,1864.0,2.0,10.0,38.0,1,NaN,NaN,Trumball,NaN,Judiciary,2016-02-25T17:29:59.732-05:00
1283,us-nara-amending-america-001284,A,"Ames, Herman V. The Proposed Amendments to the...",1135,Civil rights: Citizenship (The XIV Amendment),circa,1866.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-25T17:29:59.732-05:00
1466,us-nara-amending-america-001467,A,"Ames, Herman V. The Proposed Amendments to the...",1284,Suffrage. (The XV Amendment),NaN,1867.0,3.0,7.0,40.0,2,NaN,NaN,Henderson,Missouri,NaN,2016-02-25T17:29:59.732-05:00


In [11]:
# Group the data by year and count the number of amendments, sort by count
yearly_counts = data.groupby('year').size().reset_index(name='count').sort_values(by='count', ascending=False)
print(yearly_counts.shape)
yearly_counts

(221, 2)


,year,count
174,1969.0,616
176,1971.0,444
170,1965.0,397
172,1967.0,397
168,1963.0,378
...,...,...
11,1801.0,1
3,1791.0,1
7,1797.0,1
55,1845.0,1


In [12]:
# Group the yearly_counts by count and get the number of years for each count, then sort by count
yearly_counts_sorted = yearly_counts.groupby('count').size().reset_index(name='years').sort_values(by='count')
print(yearly_counts_sorted.shape)
yearly_counts_sorted

(98, 2)


,count,years
0,1,6
1,2,4
2,3,3
3,4,9
4,5,2
...,...,...
93,326,1
94,378,1
95,397,2
96,444,1


In [13]:
# Check if any years don't have any proposed amendments
years_with_no_amendments = set(range(1788, 2014)) - set(yearly_counts['year'])
print(f"Years with no proposed amendments: {sorted(years_with_no_amendments)}")



Years with no proposed amendments: [1792, 1796, 1847, 1855, 1856, 1857, 1859]
